###### Specifing run configurations and regularization constants. These are used to tweak the model.

In [1]:
# RUN CONFIGURATIONS
TEST = 21
CNN_ACT_F = "relu"
DNS_ACT_F = "tanh"
ARCH = f"CNN(64,128,256)_{CNN_ACT_F}_DENSE(128,64,32,12)_{DNS_ACT_F}"
IMG_SIZE = 70
N_SPLIT = 5
EPOCHS = 30
BATCH_SIZE = 50
VAL_SPLIT = 0.1

LR = 0.0095
MM = 0.9


# REGULARIZATIONS
DROPOUT_RATE = 0.5
LAMBDA_1 = 1e-5
LAMBDA_2 = 1e-6

###### Loading the dataset from pickle

In [2]:
import pickle

X = pickle.load(open(f"X_{IMG_SIZE}.pickle", "rb"))
Y = pickle.load(open(f"Y_{IMG_SIZE}.pickle", "rb"))

###### Creating the report name for the corresponding tweak of the model

In [3]:
SET_1 = f"{TEST} - {ARCH} - {IMG_SIZE}px_{EPOCHS}e_{BATCH_SIZE}bs_{int(VAL_SPLIT*100)}vs_REG_{int(DROPOUT_RATE*100)}do_{int(LAMBDA_1*1000000)}L1_{int(LAMBDA_2*1000000)}L2"
SET_2 = f"{TEST} - {IMG_SIZE}px_{EPOCHS}e_{BATCH_SIZE}bs_{int(VAL_SPLIT*100)}vs_REG_{int(DROPOUT_RATE*100)}do_{int(LAMBDA_1*1000000)}L1_{int(LAMBDA_2*1000000)}L2_{int(LR*10000)}LR_{int(MM*100)}M"
REPORT_NAME = SET_2

###### Importing tensorflow-gpu and other related libs and classes. Tensorboard is used to see live tables.

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import time

C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Thilina\.conda\envs\tensorflow-gpu-1-13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Thilina\.conda\envs\tenso

In [5]:
MODEL_NAME = f"NN_GA_CNN_{int(time.time())}"

tensorboard = TensorBoard(log_dir=f"logs/{MODEL_NAME}")

def create_cnn_model():
    model = Sequential()
    
    model.add(Conv2D(64, (3,3), input_shape=(IMG_SIZE,IMG_SIZE,1)))  # input shape is IMG_SIZExIMG_SIZEx1
#     model.add(BatchNormalization(axis=3))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
    model.add(Conv2D(128, (3,3)))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
    model.add(Conv2D(256, (3,3)))
    model.add(Activation(CNN_ACT_F))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(128, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Conv2D(256, (3,3)))
#     model.add(Activation(CNN_ACT_F))
#     model.add(MaxPooling2D(pool_size=(3,3), strides=(2, 2)))
    
#     model.add(Dropout(rate=DROPOUT_RATE))
    
    model.add(Flatten())
    
#     model.add(Dense(128))
    model.add(Dense(128, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(32, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=LAMBDA_1, l2=LAMBDA_2)))
    model.add(Activation(DNS_ACT_F))
    
    model.add(Dense(12))
    model.add(Activation("softmax"))
    
    optimizer = tf.keras.optimizers.SGD(lr=LR, momentum=MM, nesterov=False)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

###### Creating folder for saving reports of the model executions

In [6]:
import os

if not os.path.exists('reports'):
    os.makedirs('reports')

file = open(f"reports/{REPORT_NAME}.txt", "w")

###### Designed the 5-fold cross validation training and test with specific validation split. Final accuracy is averaged by the accuracies of the separate 5 folds. Time taken for the execution is also measured.

In [7]:
from sklearn.model_selection import KFold

# initializing variables for kFold run and average accuracy
current_fold = 0
sum_acc = 0
avg_acc = 0

start = time.process_time()

for train_index, test_index in KFold(N_SPLIT).split(X):
    current_fold += 1
    
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    model = create_cnn_model()
    
#     earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VAL_SPLIT, callbacks=[tensorboard])
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(f"\nFOLD-{current_fold}: Loss={val_loss} , Accuracy={val_acc}\n")
    
    sum_acc += val_acc 
    file.write(f"{current_fold}-FOLD | Loss={round(val_loss,4)},\tAccuracy={round(val_acc,4)},\tAverage_Accuracy={round(sum_acc/current_fold,4)}\n")
    
    if(current_fold == N_SPLIT):
        avg_acc = round(sum_acc/current_fold,4)
    

avg_acc_line = f"\nAverage Accuracy : {round(avg_acc,4)}"

end = time.process_time()
time_taken = f"\nExecution Time\t : {round(end-start,4)}s"

print(avg_acc_line)
print(time_taken)

file.write(avg_acc_line)
file.write(time_taken)
    
file.close()

Instructions for updating:
Colocations handled automatically by placer.
Train on 3420 samples, validate on 380 samples
Epoch 1/30
3420/3420 [==============================] - 9s 3ms/sample - loss: 2.4452 - acc: 0.1418 - val_loss: 2.4269 - val_acc: 0.1605
Epoch 2/30
3420/3420 [==============================] - 6s 2ms/sample - loss: 2.4029 - acc: 0.1515 - val_loss: 2.3819 - val_acc: 0.1658
Epoch 3/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 2.3693 - acc: 0.1550 - val_loss: 2.4054 - val_acc: 0.1658
Epoch 4/30
3420/3420 [==============================] - 8s 2ms/sample - loss: 2.3482 - acc: 0.1754 - val_loss: 2.3505 - val_acc: 0.1553
Epoch 5/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 2.3593 - acc: 0.1626 - val_loss: 2.4371 - val_acc: 0.1447
Epoch 6/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 2.3292 - acc: 0.1827 - val_loss: 2.3143 - val_acc: 0.2105
Epoch 7/30
3420/3420 [==============================] - 6s 2ms/sa

3420/3420 [==============================] - 7s 2ms/sample - loss: 0.1097 - acc: 0.9877 - val_loss: 1.6060 - val_acc: 0.5237
Epoch 27/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0687 - acc: 0.9965 - val_loss: 1.6146 - val_acc: 0.5500
Epoch 28/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0547 - acc: 0.9988 - val_loss: 1.6637 - val_acc: 0.5447
Epoch 29/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0481 - acc: 0.9997 - val_loss: 1.6570 - val_acc: 0.5447
Epoch 30/30
950/950 [==============================] - 1s 750us/sample - loss: 1.5392 - acc: 0.6095

FOLD-2: Loss=1.539151146788346 , Accuracy=0.609473705291748

Train on 3420 samples, validate on 380 samples
Epoch 1/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 2.4461 - acc: 0.1319 - val_loss: 2.4453 - val_acc: 0.1395
Epoch 2/30
3420/3420 [==============================] - 6s 2ms/sample - loss: 2.4167 - acc: 0.1518 - val_loss: 2.4073 -

3420/3420 [==============================] - 7s 2ms/sample - loss: 0.2643 - acc: 0.9360 - val_loss: 1.7128 - val_acc: 0.5316
Epoch 24/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.1595 - acc: 0.9754 - val_loss: 1.7605 - val_acc: 0.5132
Epoch 25/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.1129 - acc: 0.9854 - val_loss: 1.6866 - val_acc: 0.5289
Epoch 26/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0755 - acc: 0.9947 - val_loss: 1.7285 - val_acc: 0.5289
Epoch 27/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0572 - acc: 0.9977 - val_loss: 1.7424 - val_acc: 0.5289
Epoch 28/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0475 - acc: 0.9994 - val_loss: 1.7522 - val_acc: 0.5342
Epoch 29/30
3420/3420 [==============================] - 7s 2ms/sample - loss: 0.0421 - acc: 0.9994 - val_loss: 1.7791 - val_acc: 0.5263
Epoch 30/30
950/950 [==============================] 

###### Saving the model for later use

In [8]:
model.save(f"{REPORT_NAME}.model")

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 68, 68, 64)        640       
_________________________________________________________________
activation_28 (Activation)   (None, 68, 68, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 33, 33, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 31, 31, 128)       73856     
_________________________________________________________________
activation_29 (Activation)   (None, 31, 31, 128)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 13, 13, 256)       295168    
__________

###### To reset GPU memory after model training. But seems this doesn't work. Always need to restart the kernel and run.

In [10]:
tf.reset_default_graph()